In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from keras import backend as K 
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
K.set_image_dim_ordering('th')

In [ ]:
max_caption_len = 15
vocab_size = 20

In [ ]:
IR2 = InceptionResNetV2(include_top=False, weights='None')
IR2.add(Flatten())
IR2.add(Dense(1024, activation='relu'))
IR2.add(Dropout(0.2))
IR2.add(Dense(1024, activation='relu'))

image_model.add(RepeatVector(CONTEXT_LENGTH))
#image_model.add(RepeatVector(max_caption_len))

In [ ]:
language_model = Sequential()
language_model.add(Embedding(vocab_size, 256, input_length=max_caption_len))
language_model.add(GRU(units=128, return_sequences=True))
language_model.add(TimeDistributed(Dense(128)))

In [ ]:
model = Sequential()
model.add(Merge([image_model, language_model], mode='concat', concat_axis=-1))
model.add(GRU(256, return_sequences=False))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
img_path = 'screenshot.jpg
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [35]:
# Our html: "<HTML><BODY><center><H1>Hello World!</H1><center></BODY></HTML>"
html = ['<HTML>', '<BODY>', '<center>', '<H1>', 'Hello', ' ', 'World!', '</H1>', '</center>', '</BODY>', '</HTML>']
syntax_to_index = {}
index_to_syntax = {}
for i,word in enumerate(html):
    syntax_to_index[word] = i
    index_to_syntax[i] = word
html_input = np.array([[syntax_to_index[txt] for txt in html]])

In [36]:
print(one)

[[ 0  1  2  3  4  5  6  7  8  9 10  0  0  0  0]]


In [66]:
decoder_target_data = np.zeros((1, len(html) - 1, len(html)), dtype='float32')

In [67]:
print(decoder_target_data)

[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]]


In [68]:
for i, word in enumerate(html):
    if i > 0:
        decoder_target_data[0, i - 1, syntax_to_index[word]] = 1.

In [69]:
print(decoder_target_data)

[[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]]
